In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName("movies").config('spark.master', 'local[*]').getOrCreate()

24/07/24 10:18:30 WARN Utils: Your hostname, themjdex-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/07/24 10:18:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/24 10:18:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
actors_df = spark.read.csv('data/actors.csv', header=True, inferSchema=True)
movie_actors_df = spark.read.csv('data/movie_actors.csv', header=True, inferSchema=True)
movies_df = spark.read.csv('data/movies.csv', header=True, inferSchema=True)

In [5]:
# Создайте временные таблицы для данных о фильмах, актерах и связях между ними
actors_df.createOrReplaceTempView('actors')
movie_actors_df.createOrReplaceTempView('movie_actors')
movies_df.createOrReplaceTempView('movies')

In [6]:
spark.sql("SELECT * FROM actors LIMIT 5").show()

+--------+-------+----------+-------+
|actor_id|   name|birth_date|country|
+--------+-------+----------+-------+
|       1|Actor_1|1960-12-31| Canada|
|       2|Actor_2|1962-12-31|     UK|
|       3|Actor_3|1964-12-31|     UK|
|       4|Actor_4|1966-12-31|     UK|
|       5|Actor_5|1968-12-31|  India|
+--------+-------+----------+-------+



In [7]:
spark.sql("SELECT * FROM movie_actors LIMIT 5").show()

+--------+--------+
|movie_id|actor_id|
+--------+--------+
|       1|      25|
|      16|       5|
|       6|      16|
|      16|      11|
|      14|      21|
+--------+--------+



In [8]:
spark.sql("SELECT * FROM movies LIMIT 5").show()

+--------+-------+------+------------+-------------+
|movie_id|  title| genre|release_date|       budget|
+--------+-------+------+------------+-------------+
|       1|Movie_1|Horror|  2000-12-31|8.660058311E7|
|       2|Movie_2|Comedy|  2001-12-31|1.274740083E7|
|       3|Movie_3|Action|  2002-12-31| 1.80157747E7|
|       4|Movie_4| Drama|  2003-12-31|4.817612061E7|
|       5|Movie_5| Drama|  2004-12-31| 7.40501611E7|
+--------+-------+------+------------+-------------+



In [10]:
# Найдите топ-5 жанров по количеству фильмов
spark.sql("SELECT genre, COUNT(title) AS num_movies FROM movies GROUP BY genre ORDER BY num_movies DESC").show()

+------+----------+
| genre|num_movies|
+------+----------+
| Drama|         6|
|Action|         6|
|Comedy|         4|
|Horror|         2|
|Sci-Fi|         2|
+------+----------+



In [15]:
join_df = spark.sql("""
                    SELECT m.movie_id, m.title, m.genre, m.release_date, m.budget, a.actor_id, a.name, a.birth_date, a.country  
                    FROM movies AS m
                    JOIN movie_actors AS ma ON m.movie_id = ma.movie_id
                    JOIN actors AS a ON ma.actor_id = a.actor_id
                    """)
join_df.createOrReplaceTempView('join_table')

In [19]:
# Найдите актера с наибольшим количеством фильмов
spark.sql("SELECT name, COUNT(movie_id) AS num_movies FROM join_table GROUP BY name ORDER BY num_movies DESC LIMIT 1").show()

+--------+----------+
|    name|num_movies|
+--------+----------+
|Actor_17|         5|
+--------+----------+



In [22]:
# Подсчитайте средний бюджет фильмов по жанрам
spark.sql("SELECT genre, AVG(budget) AS avg_budget FROM join_table GROUP BY genre").show()

+------+--------------------+
| genre|          avg_budget|
+------+--------------------+
| Drama| 6.256277161307693E7|
|Horror| 8.711155335875002E7|
|Comedy| 4.883974718111112E7|
|Action|2.5901128669999998E7|
|Sci-Fi|       7.929615029E7|
+------+--------------------+



In [38]:
#Найдите фильмы, в которых снялось более одного актера из одной страны
spark.sql("SELECT title, country, COUNT(actor_id) AS cnt FROM join_table GROUP BY title, country HAVING cnt > 1").show()

+--------+---------+---+
|   title|  country|cnt|
+--------+---------+---+
| Movie_7|    India|  2|
| Movie_3|      USA|  2|
|Movie_10|       UK|  2|
|Movie_15|    India|  2|
|Movie_18|Australia|  2|
| Movie_1|    India|  3|
| Movie_2|      USA|  2|
| Movie_7|      USA|  2|
|Movie_10|      USA|  2|
+--------+---------+---+

